Okay, we need to do sanity check to make sure that we have implemented the functions in the notebook "3_preprocessing_for_metrics_other_blocks_stage_1.ipynb".

The easiest way to verify this is to use these functions on the train data (that was preprocessed using the notebook "1_preprocessing_for_seq_2_seq_stage_1.ipynb"), preprocess the data using the new functions and make sure the np arrays stored using the two methods is excatly the same.

Paste below the required codes for the functions and the model to extract features.

##### This mehtod is much fater than what was done in the "1_preprocessing_for_seq_2_seq_stage_1.ipynb" notebook

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
import tensorflow as tf
import os

2025-06-04 19:42:40.011858: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-04 19:42:40.337403: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-04 19:42:40.337464: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-04 19:42:40.407685: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-04 19:42:40.515184: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# load the finetuned gmp model
gmp_model = tf.keras.models.load_model('../../Spring_2024/Bayes_for_comps/TS_bayes_implementation_for_TN/models/trained_gmp_model_dense_32_new.h5')

2025-06-04 19:42:50.270507: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30310 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


In [3]:
gmp_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                           

In [4]:
# define the feature extractor

# feature extractor input
feat_ext_input = gmp_model.input

# feature extractor output - do this at the ReLu activation layer - as this will give the same features as the dropout layer (It does not matter if it is the dropout or the activation layer, the extracted features will be the same)
feat_ext_output = gmp_model.layers[-4].output

feature_extractor_model = tf.keras.models.Model(inputs = feat_ext_input, outputs = feat_ext_output)

In [5]:
feature_extractor_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                             

In [6]:
# Write a function for extracting onlt the horizontal images out of all the files in the location

def get_horizontal_images(file_path):
    # get all contents at the file path
    all_files = os.listdir(file_path)
    # get only the image files as we do not need the xml files for the seq2seq model
    only_images = [file for file in all_files if file.split(".")[-1] == 'jpeg']
    # from all the images only choose the horizontal ones
    horizontal_images = [file for file in only_images if file.split('.')[0][-10:] in horizontal_image_list]
    # sort the list
    horizontal_images.sort()
    # return the image list
    return horizontal_images

In [7]:
# The following alternate functions are written to improve the execution time of the prediction from the model to extract features

In [8]:
def create_sub_windows(folder_path, file, stride = 30, kernel_size = 30):
    # joined image path
    joined_im_path = os.path.join(folder_path, file)
    # read the image
    loaded_im_file = plt.imread(joined_im_path)
    # create subwindows and get prediction
    img_height = loaded_im_file.shape[0]
    img_width = loaded_im_file.shape[1]

    # catch all subwindows here
    all_subwindows = []
    # you can also keep track the subwindows here if required - but let's not worry about that for now
    for i in  range(0, img_height, stride):
        for j in range(0, img_width, stride):
            sub_window = loaded_im_file[i: i + kernel_size, j : j + kernel_size,:]
            # resize the subwindow - for 300*300
            sub_window = resize(sub_window, (kernel_size, kernel_size,3))
            # append these to the list
            all_subwindows.append(sub_window)
            
    return all_subwindows  

In [9]:
def extract_features(file_name, file_path):
    # get the subwindows
    subwindows = create_sub_windows(file_path, file_name, 30, 30)
    # stack the subwindows
    stacked_subwindows = np.stack(subwindows, axis = 0)
    # print the shape of this
    print(stacked_subwindows.shape)
    # extract features
    extracted_featrues = feature_extractor_model.predict(stacked_subwindows)
    
    return extracted_featrues

In [10]:
horizontal_image_list = ['2020_08_03', '2020_08_04', '2020_08_06', '2020_08_07', '2020_08_11', '2020_08_12', '2020_08_14', '2020_08_15', '2020_08_17', '2020_08_18', '2020_08_19', '2020_08_21', '2020_08_25', '2020_08_26', '2020_08_27', '2020_08_28', '2020_08_31', '2020_09_02', '2020_09_07', '2020_09_16']

Block 0101

In [11]:
blk_0101_data_path = '../../Spring_2024/S_lab_TasselNet/Block_1_TN/Block_1_images_and_xml'

In [12]:
# we can still use the function above to get the horizontal images
blk_0101_horizontal_images = get_horizontal_images(blk_0101_data_path)

In [13]:
# separate the inputs and the targets
blk_0101_input_time_point_images = blk_0101_horizontal_images[:13]
blk_0101_input_time_point_images.sort()
blk_0101_output_time_point_images = blk_0101_horizontal_images[-7:]
blk_0101_output_time_point_images.sort()

In [14]:
# blk_0101_input_time_point_images, blk_0101_output_time_point_images

In [15]:
%%time
# get the extracted input features

blk_0101_alt_all_features = []
for file in blk_0101_input_time_point_images:
    extracted_features = extract_features(file, blk_0101_data_path)
    blk_0101_alt_all_features.append(extracted_features)

(910, 30, 30, 3)


2025-06-04 19:42:51.455162: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


29/29 [==============================] - 2s 5ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 4.99 s, sys: 507 ms, total: 5.5 s
Wall time: 6.3 s


In [16]:
# stack the extracted features
blk_0101_stacked_extracted_input_features = np.stack(blk_0101_alt_all_features, axis = 1)

In [17]:
blk_0101_stacked_extracted_input_features.shape

(910, 13, 32)

In [18]:
# check if mean matches
np.mean(blk_0101_stacked_extracted_input_features)

0.3515611

In [19]:
# Store this stack? 
np.save("seq_2_seq_train_data_alt/block_0101_extracted_input_features.npy", blk_0101_stacked_extracted_input_features)

In [20]:
# extract the targets

In [21]:
%%time
# get the extracted targets

blk_0101_targets = []
for file in blk_0101_output_time_point_images:
    extracted_features = extract_features(file, blk_0101_data_path)
    blk_0101_targets.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 2.38 s, sys: 138 ms, total: 2.52 s
Wall time: 2.4 s


In [22]:
# stack the extracted features
blk_0101_stacked_extracted_targets = np.stack(blk_0101_targets, axis = 1)

In [23]:
blk_0101_stacked_extracted_targets.shape

(910, 7, 32)

In [24]:
# Store this stack
np.save("seq_2_seq_train_data_alt/block_0101_extracted_target_features.npy", blk_0101_stacked_extracted_targets)

In [25]:
# make sure these values are the same
# load the previously stored np values

all_train_data_previously_preprocessed = os.listdir("seq_2_seq_train_data")

In [26]:
all_train_data_previously_preprocessed.sort()
all_train_data_previously_preprocessed

['block_0101_extracted_input_features.npy',
 'block_0101_extracted_target_features.npy',
 'block_0102_extracted_input_features.npy',
 'block_0102_extracted_target_features.npy',
 'block_0203_extracted_input_features.npy',
 'block_0203_extracted_target_features.npy',
 'block_0301_extracted_input_features.npy',
 'block_0301_extracted_target_features.npy']

In [27]:
# load these data
alt_0101_input_features = np.load(os.path.join("seq_2_seq_train_data", "block_0101_extracted_input_features.npy"))
alt_0101_targets = np.load(os.path.join("seq_2_seq_train_data", "block_0101_extracted_target_features.npy"))

In [28]:
print(np.mean(np.round(blk_0101_stacked_extracted_input_features, 3) == np.round(alt_0101_input_features,3)))
print(np.mean(np.round(blk_0101_stacked_extracted_targets,3) == np.round(alt_0101_targets,3)))

0.9999445266272189
0.9999607535321821


In [29]:
np.mean(blk_0101_stacked_extracted_input_features), np.mean(alt_0101_input_features)

(0.3515611, 0.3515611)

In [30]:
np.mean(blk_0101_stacked_extracted_targets), np.mean(alt_0101_targets)

(0.36970586, 0.36970586)

In [31]:
# seems they are correct, do this for the rest of the train data as well

Block 0102

In [32]:
blk_0102_data_path = '../../Spring_2024/S_lab_TasselNet/Block_2_TN/Block_2_images_and_xml'

In [33]:
# we can still use the function above to get the horizontal images
blk_0102_horizontal_images = get_horizontal_images(blk_0102_data_path)

In [34]:
# separate the inputs and the targets
blk_0102_input_time_point_images = blk_0102_horizontal_images[:13]
blk_0102_input_time_point_images.sort()
blk_0102_output_time_point_images = blk_0102_horizontal_images[-7:]
blk_0102_output_time_point_images.sort()

In [35]:
# blk_0102_input_time_point_images, blk_0102_output_time_point_images

In [36]:
%%time
# get the extracted input features

blk_0102_alt_all_features = []
for file in blk_0102_input_time_point_images:
    extracted_features = extract_features(file, blk_0102_data_path)
    blk_0102_alt_all_features.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 4.44 s, sys: 261 ms, total: 4.7 s
Wall time: 4.5 s


In [37]:
# stack the extracted features
blk_0102_stacked_extracted_input_features = np.stack(blk_0102_alt_all_features, axis = 1)

In [38]:
blk_0102_stacked_extracted_input_features.shape

(910, 13, 32)

In [39]:
# check if mean matches
np.mean(blk_0102_stacked_extracted_input_features)

0.35029814

In [40]:
# Store this stack? 
np.save("seq_2_seq_train_data_alt/block_0102_extracted_input_features.npy", blk_0102_stacked_extracted_input_features)

In [41]:
# extract the targets

In [42]:
%%time
# get the extracted targets

blk_0102_targets = []
for file in blk_0102_output_time_point_images:
    extracted_features = extract_features(file, blk_0102_data_path)
    blk_0102_targets.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 2.4 s, sys: 80.1 ms, total: 2.48 s
Wall time: 2.36 s


In [43]:
# stack the extracted features
blk_0102_stacked_extracted_targets = np.stack(blk_0102_targets, axis = 1)

In [44]:
blk_0102_stacked_extracted_targets.shape

(910, 7, 32)

In [45]:
# Store this stack
np.save("seq_2_seq_train_data_alt/block_0102_extracted_target_features.npy", blk_0102_stacked_extracted_targets)

In [46]:
# load these data
alt_0102_input_features = np.load(os.path.join("seq_2_seq_train_data", "block_0102_extracted_input_features.npy"))
alt_0102_targets = np.load(os.path.join("seq_2_seq_train_data", "block_0102_extracted_target_features.npy"))

In [47]:
print(np.mean(np.round(blk_0102_stacked_extracted_input_features, 3) == np.round(alt_0102_input_features,3)))
print(np.mean(np.round(blk_0102_stacked_extracted_targets,3) == np.round(alt_0102_targets,3)))

0.999947168216399
0.9999460361067504


In [48]:
np.mean(blk_0102_stacked_extracted_input_features), np.mean(alt_0102_input_features)

(0.35029814, 0.35029814)

In [49]:
np.mean(blk_0102_stacked_extracted_targets), np.mean(alt_0102_targets)

(0.3753665, 0.37536648)

Block 0203

In [50]:
blk_0203_data_path = '../../Spring_2024/S_lab_TasselNet/Block_9_TN/Block_9_images_and_xml'

In [51]:
# we can still use the function above to get the horizontal images
blk_0203_horizontal_images = get_horizontal_images(blk_0203_data_path)

In [52]:
# separate the inputs and the targets
blk_0203_input_time_point_images = blk_0203_horizontal_images[:13]
blk_0203_input_time_point_images.sort()
blk_0203_output_time_point_images = blk_0203_horizontal_images[-7:]
blk_0203_output_time_point_images.sort()

In [53]:
# blk_0203_input_time_point_images, blk_0203_output_time_point_images

In [54]:
%%time
# get the extracted input features

blk_0203_alt_all_features = []
for file in blk_0203_input_time_point_images:
    extracted_features = extract_features(file, blk_0203_data_path)
    blk_0203_alt_all_features.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 4.43 s, sys: 204 ms, total: 4.63 s
Wall time: 4.42 s


In [55]:
# stack the extracted features
blk_0203_stacked_extracted_input_features = np.stack(blk_0203_alt_all_features, axis = 1)

In [56]:
blk_0203_stacked_extracted_input_features.shape

(910, 13, 32)

In [57]:
# check if mean matches
np.mean(blk_0203_stacked_extracted_input_features)

0.34053323

In [58]:
# Store this stack? 
np.save("seq_2_seq_train_data_alt/block_0203_extracted_input_features.npy", blk_0203_stacked_extracted_input_features)

In [59]:
# extract the targets

In [60]:
%%time
# get the extracted targets

blk_0203_targets = []
for file in blk_0203_output_time_point_images:
    extracted_features = extract_features(file, blk_0203_data_path)
    blk_0203_targets.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 2.39 s, sys: 123 ms, total: 2.51 s
Wall time: 2.4 s


In [61]:
# stack the extracted features
blk_0203_stacked_extracted_targets = np.stack(blk_0203_targets, axis = 1)

In [62]:
blk_0203_stacked_extracted_targets.shape

(910, 7, 32)

In [63]:
# Store this stack
np.save("seq_2_seq_train_data_alt/block_0203_extracted_target_features.npy", blk_0203_stacked_extracted_targets)

In [64]:
# load these data
alt_0203_input_features = np.load(os.path.join("seq_2_seq_train_data", "block_0203_extracted_input_features.npy"))
alt_0203_targets = np.load(os.path.join("seq_2_seq_train_data", "block_0203_extracted_target_features.npy"))

In [65]:
print(np.mean(np.round(blk_0203_stacked_extracted_input_features, 3) == np.round(alt_0203_input_features,3)))
print(np.mean(np.round(blk_0203_stacked_extracted_targets,3) == np.round(alt_0203_targets,3)))

0.9999207523245984
0.999916601255887


In [66]:
np.mean(blk_0203_stacked_extracted_input_features), np.mean(alt_0203_input_features)

(0.34053323, 0.3405332)

In [67]:
np.mean(blk_0203_stacked_extracted_targets), np.mean(alt_0203_targets)

(0.33241686, 0.33241686)

Block 0301

In [68]:
blk_0301_data_path = '../../Spring_2024/S_lab_TasselNet/Block_13_TN/Block_13_images_and_xml'

In [69]:
# we can still use the function above to get the horizontal images
blk_0301_horizontal_images = get_horizontal_images(blk_0301_data_path)

In [70]:
# separate the inputs and the targets
blk_0301_input_time_point_images = blk_0301_horizontal_images[:13]
blk_0301_input_time_point_images.sort()
blk_0301_output_time_point_images = blk_0301_horizontal_images[-7:]
blk_0301_output_time_point_images.sort()

In [71]:
# blk_0301_input_time_point_images, blk_0301_output_time_point_images

In [72]:
%%time
# get the extracted input features

blk_0301_alt_all_features = []
for file in blk_0301_input_time_point_images:
    extracted_features = extract_features(file, blk_0301_data_path)
    blk_0301_alt_all_features.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
CPU times: user 4.49 s, sys: 230 ms, total: 4.72 s
Wall time: 4.5 s


In [73]:
# stack the extracted features
blk_0301_stacked_extracted_input_features = np.stack(blk_0301_alt_all_features, axis = 1)

In [74]:
blk_0301_stacked_extracted_input_features.shape

(910, 13, 32)

In [75]:
# check if mean matches
np.mean(blk_0301_stacked_extracted_input_features)

0.36407468

In [76]:
# Store this stack? 
np.save("seq_2_seq_train_data_alt/block_0301_extracted_input_features.npy", blk_0301_stacked_extracted_input_features)

In [77]:
# extract the targets

In [78]:
%%time
# get the extracted targets

blk_0301_targets = []
for file in blk_0301_output_time_point_images:
    extracted_features = extract_features(file, blk_0301_data_path)
    blk_0301_targets.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 3ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 2.59 s, sys: 186 ms, total: 2.78 s
Wall time: 2.65 s


In [79]:
# stack the extracted features
blk_0301_stacked_extracted_targets = np.stack(blk_0301_targets, axis = 1)

In [80]:
blk_0301_stacked_extracted_targets.shape

(910, 7, 32)

In [81]:
# Store this stack
np.save("seq_2_seq_train_data_alt/block_0301_extracted_target_features.npy", blk_0301_stacked_extracted_targets)

In [82]:
# load these data
alt_0301_input_features = np.load(os.path.join("seq_2_seq_train_data", "block_0301_extracted_input_features.npy"))
alt_0301_targets = np.load(os.path.join("seq_2_seq_train_data", "block_0301_extracted_target_features.npy"))

In [83]:
print(np.mean(np.round(blk_0301_stacked_extracted_input_features, 3) == np.round(alt_0301_input_features,3)))
print(np.mean(np.round(blk_0301_stacked_extracted_targets,3) == np.round(alt_0301_targets,3)))

0.9999445266272189
0.9999215070643642


In [84]:
np.mean(blk_0301_stacked_extracted_input_features), np.mean(alt_0301_input_features)

(0.36407468, 0.36407468)

In [85]:
np.mean(blk_0301_stacked_extracted_targets), np.mean(alt_0301_targets)

(0.36661348, 0.36661348)

In [86]:
# We should do this for validation data too

Block 0204

In [87]:
blk_0204_data_path = '../../Spring_2024/S_lab_TasselNet/Block_10_TN/Block_10_images_and_xml'

In [88]:
# we can still use the function above to get the horizontal images
blk_0204_horizontal_images = get_horizontal_images(blk_0204_data_path)

In [89]:
# separate the inputs and the targets
blk_0204_input_time_point_images = blk_0204_horizontal_images[:13]
blk_0204_input_time_point_images.sort()
blk_0204_output_time_point_images = blk_0204_horizontal_images[-7:]
blk_0204_output_time_point_images.sort()

In [90]:
# blk_0204_input_time_point_images, blk_0204_output_time_point_images

In [91]:
%%time
# get the extracted input features

blk_0204_alt_all_features = []
for file in blk_0204_input_time_point_images:
    extracted_features = extract_features(file, blk_0204_data_path)
    blk_0204_alt_all_features.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 4.49 s, sys: 162 ms, total: 4.66 s
Wall time: 4.44 s


In [92]:
# stack the extracted features
blk_0204_stacked_extracted_input_features = np.stack(blk_0204_alt_all_features, axis = 1)

In [93]:
blk_0204_stacked_extracted_input_features.shape

(910, 13, 32)

In [94]:
# check if mean matches
np.mean(blk_0204_stacked_extracted_input_features)

0.3899464

In [95]:
# Store this stack? 
np.save("seq_2_seq_valid_data_alt/block_0204_extracted_input_features.npy", blk_0204_stacked_extracted_input_features)

In [96]:
# extract the targets

In [97]:
%%time
# get the extracted targets

blk_0204_targets = []
for file in blk_0204_output_time_point_images:
    extracted_features = extract_features(file, blk_0204_data_path)
    blk_0204_targets.append(extracted_features)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 2.41 s, sys: 94.5 ms, total: 2.5 s
Wall time: 2.39 s


In [98]:
# stack the extracted features
blk_0204_stacked_extracted_targets = np.stack(blk_0204_targets, axis = 1)

In [99]:
blk_0204_stacked_extracted_targets.shape

(910, 7, 32)

In [100]:
# Store this stack
np.save("seq_2_seq_valid_data_alt/block_0204_extracted_target_features.npy", blk_0204_stacked_extracted_targets)

In [101]:
# load these data
alt_0204_input_features = np.load(os.path.join("seq_2_seq_valid_data", "block_0204_extracted_input_features.npy"))
alt_0204_targets = np.load(os.path.join("seq_2_seq_valid_data", "block_0204_extracted_target_features.npy"))

In [102]:
print(np.mean(np.round(blk_0204_stacked_extracted_input_features, 3) == np.round(alt_0204_input_features,3)))
print(np.mean(np.round(blk_0204_stacked_extracted_targets,3) == np.round(alt_0204_targets,3)))

0.9999418850380389
0.9999754709576139


In [103]:
np.mean(blk_0204_stacked_extracted_input_features), np.mean(alt_0204_input_features)

(0.3899464, 0.38994637)

In [104]:
np.mean(blk_0204_stacked_extracted_targets), np.mean(alt_0204_targets)

(0.36944777, 0.36944777)